## **Unifying and Combining the final data**

### 1. Unifying the data
- The goal of this step is to unify the data from strucutured and unstructured data.
    - Processing the structured data (XY)
    - Processing the embedding of the unstructured data
    - Saving...
    
### 2. Combining and preparing Y
- The goal here is to combine the data so that we end with the following information:
    - **Z**: base features
    - **ZS**: base features + MeSH-based symptoms
    - **T0**: original text
    - **T1**: t5-small
    - **T2**: bart large cnn
    - **T3**: medical summarization

    - **ZST0**: base + MeSH + original text
    - **ZST1**: base + MeSH + t5-small
    - **ZST2**: base + MeSH + bart large cnn
    - **ZST3**: base + MeSH + medical summarization

In [3]:
import os
import pickle
import pandas as pd
import numpy as np
import ast

def load_pickle(thePath):
    with open(thePath, 'rb') as f:
        data = pickle.load(f)
    return data

def save_pickle(L, given_path, file_name):
    # Ensure the directory exists
    if not os.path.exists(given_path):
        os.makedirs(given_path)
        print(f'\tDirectory created: {given_path}')
    
    # Save the list as a pickle file
    print(f'\tSaving to {given_path}/{file_name}.pkl')
    with open(os.path.join(given_path, f'{file_name}.pkl'), 'wb') as file:
        pickle.dump(L, file)

In [7]:
for embedding_model in ['bioclinicalbert', 'clinicalbert', 'gatortron']:

    the_path   = '../../../Data'
    df_emb = pd.read_csv(f'{the_path}/unstructured/emb/merged_embeddings_{embedding_model}.csv')
    print(df_emb.head())

    HADM_ID  SUBJECT_ID                                           EMB_TEXT  \
0  100001.0     58526.0  [-0.0687888041138649, -0.0074312034994363785, ...   
1  100003.0     54610.0  [0.11134158819913864, 0.23517052829265594, -0....   
2  100006.0      9895.0  [-0.1806609332561493, 0.1893262267112732, -0.6...   
3  100007.0         NaN  [-0.19414110481739044, 0.12399192154407501, -0...   
4  100009.0       533.0  [0.10732107609510422, -0.08329793065786362, -0...   

                             EMB_1_t5_small2_SUMMARY  \
0  [0.19394947588443756, 0.28114083409309387, 0.0...   
1  [0.22276514768600464, 0.11342902481555939, 0.0...   
2  [0.04641145095229149, -0.1912071704864502, -0....   
3  [0.0688566341996193, 0.36706528067588806, -0.5...   
4  [0.06423553079366684, -0.01891227997839451, -0...   

                        EMB_3_bart_large_cnn_SUMMARY  \
0  [-0.5926626324653625, 0.5007713437080383, -0.5...   
1  [0.06475762277841568, 0.1168421283364296, -0.3...   
2  [-0.14331874251365662, 

In [8]:
df_emb

,HADM_ID,SUBJECT_ID,EMB_TEXT,EMB_1_t5_small2_SUMMARY,EMB_3_bart_large_cnn_SUMMARY,EMB_4_medical_summarization_SUMMARY
0,100001.0,58526.0,"[-0.02099149115383625, -0.13575148582458496, -...","[-0.7677536606788635, -0.15218649804592133, -5...","[-0.23178930580615997, -0.6517142653465271, 0....","[-0.33415573835372925, 0.21793755888938904, -0..."
1,100003.0,54610.0,"[-0.5665531754493713, 0.029006456956267357, 0....","[-0.6021921634674072, -0.2753641903400421, -0....","[-0.34302130341529846, 0.15390759706497192, -0...","[-0.4268856644630432, 0.03696601092815399, -0...."
2,100006.0,9895.0,"[-0.039124682545661926, 0.21584393084049225, 0...","[-0.44044867157936096, 0.19246906042099, -0.40...","[-0.798671543598175, -0.03370685875415802, -0....","[-0.39929160475730896, 0.11379197239875793, -0..."
3,100007.0,NaN,"[-0.30916962027549744, 0.09940914064645767, -0...","[-0.36506378650665283, -0.010132744908332825, ...","[-0.4583309292793274, 0.36172816157341003, -0....","[-0.4435521066188812, 0.04495523124933243, -0...."
4,100009.0,533.0,"[-0.021947696805000305, -0.3520335257053375, 0...","[-0.004350616596639156, 0.009855604730546474, ...","[-1.097956657409668, 0.31021174788475037, -0.5...","[-0.2243950515985489, 0.08260001987218857, -0...."
...,...,...,...,...,...,...
42137,199993.0,20785.0,"[-0.007985936477780342, -0.04388640448451042, ...","[-0.21690726280212402, -0.10595705360174179, -...","[-0.6364158987998962, -0.003846339415758848, -...","[-0.6610079407691956, -0.06937868893146515, 0...."
42138,199994.0,23761.0,"[-0.1891549527645111, 0.28288784623146057, 0.0...","[-0.37506288290023804, -0.23252500593662262, -...","[-0.2889753580093384, 0.034182436764240265, -0...","[-0.6016339063644409, -0.271906316280365, -0.1..."
42139,199995.0,19412.0,"[0.09213720262050629, 0.15158449113368988, 0.1...","[-0.4005696177482605, -0.14899607002735138, -0...","[-0.6182138919830322, 0.022340204566717148, -0...","[-0.4742523431777954, -0.10230163484811783, -0..."
42140,199998.0,27200.0,"[-0.12438803911209106, 0.3712233603000641, -0....","[-0.6838393807411194, -0.287038117647171, -0.4...","[-0.281541109085083, 0.17265687882900238, -0.6...","[-0.19659195840358734, -0.26548147201538086, -..."


In [10]:
new_df.drop(columns=['HADM_ID'])

,SUBJECT_ID,EMB_TEXT,EMB_1_t5_small2_SUMMARY,EMB_3_bart_large_cnn_SUMMARY,EMB_4_medical_summarization_SUMMARY
0,58526.0,"[-0.0687888041138649, -0.0074312034994363785, ...","[0.19394947588443756, 0.28114083409309387, 0.0...","[-0.5926626324653625, 0.5007713437080383, -0.5...","[0.0919901430606842, 0.008501248434185982, 0.0..."
1,54610.0,"[0.11134158819913864, 0.23517052829265594, -0....","[0.22276514768600464, 0.11342902481555939, 0.0...","[0.06475762277841568, 0.1168421283364296, -0.3...","[0.06370395421981812, -0.03363833203911781, -0..."
2,9895.0,"[-0.1806609332561493, 0.1893262267112732, -0.6...","[0.04641145095229149, -0.1912071704864502, -0....","[-0.14331874251365662, 0.11323689669370651, -0...","[-0.11868766695261002, -0.09611823409795761, -..."
3,NaN,"[-0.19414110481739044, 0.12399192154407501, -0...","[0.0688566341996193, 0.36706528067588806, -0.5...","[0.1457720547914505, 0.14944635331630707, -0.4...","[0.2473159283399582, -0.0019324407912790775, -..."
4,533.0,"[0.10732107609510422, -0.08329793065786362, -0...","[0.06423553079366684, -0.01891227997839451, -0...","[-0.15307816863059998, 0.16436821222305298, -0...","[0.15557511150836945, 0.0681350976228714, -0.0..."
...,...,...,...,...,...
42137,20785.0,"[-0.1650465428829193, 0.166372150182724, -0.31...","[0.03791189193725586, 0.18803027272224426, -0....","[-0.037595346570014954, 0.1620524674654007, -0...","[0.026669379323720932, -0.11393618583679199, -..."
42138,23761.0,"[0.09483946859836578, 0.11379341781139374, -0....","[-0.01555964257568121, 0.14536091685295105, -0...","[0.05065467953681946, 0.18819169700145721, -0....","[0.20325714349746704, -0.0025444405619055033, ..."
42139,19412.0,"[-0.03536660224199295, 0.2804297208786011, -0....","[0.2780686318874359, -0.05551600083708763, 0.0...","[0.09933841973543167, 0.10636113584041595, 0.1...","[0.2312140166759491, -0.07751420140266418, 0.0..."
42140,27200.0,"[0.09091862291097641, 0.372348815202713, -0.23...","[0.16780157387256622, 0.08285443484783173, 0.0...","[0.28862524032592773, -0.014892601408064365, -...","[0.24747422337532043, -0.12266828864812851, 0...."


In [11]:
new_df

,HADM_ID,SUBJECT_ID,EMB_TEXT,EMB_1_t5_small2_SUMMARY,EMB_3_bart_large_cnn_SUMMARY,EMB_4_medical_summarization_SUMMARY
0,100001.0,58526.0,"[-0.0687888041138649, -0.0074312034994363785, ...","[0.19394947588443756, 0.28114083409309387, 0.0...","[-0.5926626324653625, 0.5007713437080383, -0.5...","[0.0919901430606842, 0.008501248434185982, 0.0..."
1,100003.0,54610.0,"[0.11134158819913864, 0.23517052829265594, -0....","[0.22276514768600464, 0.11342902481555939, 0.0...","[0.06475762277841568, 0.1168421283364296, -0.3...","[0.06370395421981812, -0.03363833203911781, -0..."
2,100006.0,9895.0,"[-0.1806609332561493, 0.1893262267112732, -0.6...","[0.04641145095229149, -0.1912071704864502, -0....","[-0.14331874251365662, 0.11323689669370651, -0...","[-0.11868766695261002, -0.09611823409795761, -..."
3,100007.0,NaN,"[-0.19414110481739044, 0.12399192154407501, -0...","[0.0688566341996193, 0.36706528067588806, -0.5...","[0.1457720547914505, 0.14944635331630707, -0.4...","[0.2473159283399582, -0.0019324407912790775, -..."
4,100009.0,533.0,"[0.10732107609510422, -0.08329793065786362, -0...","[0.06423553079366684, -0.01891227997839451, -0...","[-0.15307816863059998, 0.16436821222305298, -0...","[0.15557511150836945, 0.0681350976228714, -0.0..."
...,...,...,...,...,...,...
42137,199993.0,20785.0,"[-0.1650465428829193, 0.166372150182724, -0.31...","[0.03791189193725586, 0.18803027272224426, -0....","[-0.037595346570014954, 0.1620524674654007, -0...","[0.026669379323720932, -0.11393618583679199, -..."
42138,199994.0,23761.0,"[0.09483946859836578, 0.11379341781139374, -0....","[-0.01555964257568121, 0.14536091685295105, -0...","[0.05065467953681946, 0.18819169700145721, -0....","[0.20325714349746704, -0.0025444405619055033, ..."
42139,199995.0,19412.0,"[-0.03536660224199295, 0.2804297208786011, -0....","[0.2780686318874359, -0.05551600083708763, 0.0...","[0.09933841973543167, 0.10636113584041595, 0.1...","[0.2312140166759491, -0.07751420140266418, 0.0..."
42140,199998.0,27200.0,"[0.09091862291097641, 0.372348815202713, -0.23...","[0.16780157387256622, 0.08285443484783173, 0.0...","[0.28862524032592773, -0.014892601408064365, -...","[0.24747422337532043, -0.12266828864812851, 0...."


In [4]:
the_path   = '../../../Data'

X_B  = load_pickle(f'{the_path}/structured/X_B.pkl')
X_D  = load_pickle(f'{the_path}/structured/X_D.pkl')
X_L  = load_pickle(f'{the_path}/structured/X_L.pkl')
X_M  = load_pickle(f'{the_path}/structured/X_M.pkl')
X_P  = load_pickle(f'{the_path}/structured/X_P.pkl')
X_S  = load_pickle(f'{the_path}/structured/X_S.pkl')
VIY = load_pickle(f'{the_path}/structured/VIY.pkl')

for embedding_model in ['bioclinicalbert', 'clinicalbert', 'gatortron']:
    
    # ... unstructured data
    df_emb = pd.read_csv(f'{the_path}/unstructured/emb/merged_embeddings_{embedding_model}.csv')
    # =========================================================================
    # creating new X_VI
    X_VI2 = []
    for v,_, i in VIY:
        X_VI2.append([int(v), i])

    X_VI2 = np.array(X_VI2)
    X_VI2
    # =========================================================================
    common_HADM_ID = np.intersect1d(df_emb['HADM_ID'], X_VI2[:, 0])
    common_HADM_ID = np.sort(common_HADM_ID)
    # =========================================================================
    # Create a mask for the original array
    mask = np.in1d(X_VI2[:,0], common_HADM_ID)

    # Create a new array with the common HADM_ID
    new_X_VI2 = X_VI2[mask]

    # Sort the new array based on the common HADM_ID
    idx = np.argsort(new_X_VI2[:, 0])
    new_X_VI2 = new_X_VI2[idx]

    print(X_VI2.shape, new_X_VI2.shape)
    new_X_VI2

    # =========================================================================
    X_B2 = X_B[mask][idx]
    X_D2 = X_D[mask][idx]
    X_L2 = X_L[mask][idx]
    X_M2 = X_M[mask][idx]
    X_P2 = X_P[mask][idx]
    X_S2 = X_S[mask][idx]

    VIY2 = VIY[mask][idx]
    # =========================================================================
    # Create a mask for the original DataFrame
    df = df_emb.copy()
    mask_df = df['HADM_ID'].isin(common_HADM_ID)

    # Create a new DataFrame with the common HADM_ID
    new_df = df[mask_df].copy()

    # Sort the new DataFrame based on the common HADM_ID
    new_df = new_df.set_index('HADM_ID')
    new_df = new_df.loc[common_HADM_ID]
    new_df = new_df.reset_index()

    # =========================================================================
    from datasets import Dataset
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import LabelEncoder

    # Helper Functions
    def classify_los_3_classes(los_list):
        return [0 if los < 3 else 1 if 3 <= los <= 7 else 2 for los in los_list]

    # Encode Labels
    labels = list(classify_los_3_classes(VIY2[:, 2]))
    visits = list(VIY2[:, 0])
    label_encoder = LabelEncoder()
    labels = label_encoder.fit_transform(labels)

    E = new_df.drop(columns=['HADM_ID']).values

    # Ensure structured data is converted to lists (if needed)
    XM_list = X_M2.tolist()
    XS_list = X_S2.tolist()
    XB_list = X_B2.tolist()
    XD_list = X_D2.tolist()
    XL_list = X_L2.tolist()
    XP_list = X_P2.tolist()

    # Perform the train-test split
    (train_labels, test_labels, 
    train_visits, test_visits, 
    train_XM, test_XM, 
    train_XS, test_XS,
    train_XB, test_XB,
    train_XD, test_XD,
    train_XL, test_XL,
    train_XP, test_XP,
    train_E , test_E) = train_test_split(labels, visits, XM_list, XS_list, XB_list, XD_list, XL_list, XP_list, E,test_size=0.2, random_state=42)#, stratify=labels)

    train_dataset = Dataset.from_dict({"label": train_labels, 
                                    "HADM_ID": train_visits})

    train_dataset2 = Dataset.from_dict({"label": train_labels, 
                                    "HADM_ID": train_visits,
                                    "XM": train_XM,
                                    "XS": train_XS,
                                    "XB": train_XB,
                                    "XD": train_XD,
                                    "XL": train_XL,
                                    "XP": train_XP,
                                    "XE": train_E})

    test_dataset  = Dataset.from_dict({"label": test_labels,  
                                    "HADM_ID": test_visits})

    test_dataset2  = Dataset.from_dict({"label": test_labels,  
                                    "HADM_ID": test_visits,
                                    "XM": test_XM,
                                    "XS": test_XS,
                                    "XB": test_XB,
                                    "XD": test_XD,
                                    "XL": test_XL,
                                    "XP": test_XP,
                                    "XE": test_E})


    saving_path = f'{the_path}/training/bert2_no_stratify/{embedding_model}'
    # saving_path = f'{the_path}/training/bert2_no_stratify'
    os.makedirs(saving_path, exist_ok=True)  # Create directory if it doesn't exist

    # train_dataset.save_to_disk(f"{saving_path}/train_dataset_text")
    # test_dataset.save_to_disk(f"{saving_path}/test_dataset_text")

    train_dataset2.save_to_disk(f"{saving_path}/train_dataset_X")
    test_dataset2.save_to_disk(f"{saving_path}/test_dataset_X")
    print("Datasets saved successfully!")

    # =========================================================================
    # =========================================================================
    # =========================================================================


(44038, 2) (42142, 2)


/lustre/home/almusawiaf/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ArrowInvalid: Could not convert '[0.08804028481245041, 3.212116644135676e-05, -0.23575040698051453, 0.16293558478355408, -0.28895261883735657, -0.3358992636203766, 0.15519391000270844, 0.03240995109081268, 0.23018696904182434, -0.3842872083187103, -0.5740727186203003, 0.4144507944583893, -0.062411900609731674, 0.2796943485736847, -0.49740827083587646, 0.0446215383708477, -0.2786431908607483, -0.14831937849521637, 0.18244977295398712, 0.3042297065258026, -0.04639073833823204, -0.07114309817552567, -0.5122264623641968, -0.37687331438064575, -0.40697211027145386, -0.4610646665096283, 0.6062809824943542, 1.014302134513855, 0.06579133868217468, 0.6500542163848877, 0.07698780298233032, 0.5431334972381592, -0.044028207659721375, -0.18801791965961456, -0.37457773089408875, -0.09031741321086884, 0.1764967292547226, -0.07297398149967194, 0.11481375992298126, 0.38808000087738037, -0.2821698784828186, -0.021165624260902405, 0.4237474501132965, 0.07699337601661682, 0.4385824501514435, -0.0886222869